In [1]:
import os
import pandas as pd
from datetime import date
from pyprojroot import here
from dotenv import load_dotenv

In [2]:
# import environment variables
load_dotenv()
MOVIES_FROM_YEAR=pd.to_numeric(os.environ['MOVIES_FROM_YEAR'])
MOVIES_TO_YEAR=pd.to_numeric(os.environ['MOVIES_TO_YEAR'])
URL_ACTOR_BASICS=os.environ['URL_ACTOR_BASICS']
URL_TITLE_AKAS=os.environ['URL_TITLE_AKAS']
URL_TITLE_BASICS=os.environ['URL_TITLE_BASICS']
URL_TITLE_PRINCIPAL=os.environ['URL_TITLE_PRINCIPAL']
URL_TITLE_RATING=os.environ['URL_TITLE_RATING']


In [7]:
def save_movie_title_to_csv(df_tosave, fileName):
    df_tosave.to_csv(here("data\\" + fileName))

In [4]:
df_titles = pd.read_csv(
    URL_TITLE_BASICS, compression='gzip', 
    sep='\t', index_col='tconst', usecols=[
        'tconst', 'titleType', 
        'originalTitle', 'isAdult', 
        'startYear', 'runtimeMinutes', 
        'genres'],
        dtype={
        'tconst' : 'string',
        'titleType' : 'string',
        'originalTitle' : 'string',
        'isAdult' : 'string',
        'startYear':'string',
        'runtimeMinutes' : 'string',
        'genres' : 'string'

    })

df_titles = df_titles[(df_titles.titleType == "movie")]
df_titles["isAdult"] = df_titles["isAdult"].replace("\\N", "")
df_titles["genres"] = df_titles["genres"].replace("\\N", "")
df_titles["startYear"] = df_titles["startYear"].replace("\\N", "0")
df_titles["runtimeMinutes"] = df_titles["runtimeMinutes"].replace("\\N", "")
df_titles["runtimeMinutes"] = pd.to_numeric(df_titles['runtimeMinutes'], errors='coerce')
df_titles["startYear"] = pd.to_numeric(df_titles['startYear'], errors='coerce')

updatedtype = {
    'startYear': int,
    'runtimeMinutes': float
}

df_titles.astype(updatedtype)

,titleType,originalTitle,isAdult,startYear,runtimeMinutes,genres
tconst,,,,,,
tt0000009,movie,Miss Jerry,0,1894,45.0,Romance
tt0000147,movie,The Corbett-Fitzsimmons Fight,0,1897,100.0,"Documentary,News,Sport"
tt0000502,movie,Bohemios,0,1905,100.0,
tt0000574,movie,The Story of the Kelly Gang,0,1906,70.0,"Action,Adventure,Biography"
tt0000591,movie,L'enfant prodigue,0,1907,90.0,Drama
...,...,...,...,...,...,...
tt9916622,movie,Rodolpho Teóphilo - O Legado de um Pioneiro,0,2015,57.0,Documentary
tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,0,2007,100.0,Documentary
tt9916706,movie,Dankyavar Danka,0,2013,NaN,Comedy


In [5]:
df_title_akas = []
df_title_akas = pd.read_csv(URL_TITLE_AKAS, compression='gzip', sep='\t', dtype={
    'titleId': 'string',
    'ordering': 'int',
    'title': 'string',
    'region': 'string',
    'language': 'string',
    'types': 'string',
    'attributes': 'string',
    'isOriginalTitle': 'bool'
}, index_col="titleId")

In [6]:
df_title_akas = df_title_akas[(df_title_akas.index.isin(df_titles.index))]
df_title_akas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3386534 entries, tt0000009 to tt9916754
Data columns (total 7 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   ordering         int64 
 1   title            string
 2   region           string
 3   language         string
 4   types            string
 5   attributes       string
 6   isOriginalTitle  bool  
dtypes: bool(1), int64(1), string(5)
memory usage: 184.1 MB


In [7]:
df_title_ratings = []
df_title_ratings = pd.read_csv(URL_TITLE_RATING, compression='gzip', sep='\t', dtype={
    'tconst' : 'string',
    'averageRating'	: 'string',
    'numVotes' : 'string'
}, index_col='tconst')

df_titles["averageRating"] = df_titles["runtimeMinutes"].replace("\\N", "")
df_titles["averageRating"] = pd.to_numeric(df_titles['runtimeMinutes'], errors='coerce')
df_titles["numVotes"] = df_titles["runtimeMinutes"].replace("\\N", "")
df_titles["numVotes"] = pd.to_numeric(df_titles['runtimeMinutes'], errors='coerce')
df_title_ratings.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1495761 entries, tt0000001 to tt9916880
Data columns (total 2 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   averageRating  1495761 non-null  string
 1   numVotes       1495761 non-null  string
dtypes: string(2)
memory usage: 34.2 MB


In [8]:
df_titles = df_titles[(df_titles.index.isin(df_title_ratings.index))]

df_titles['averageRating'] = (df_titles.index.map(lambda x: df_title_ratings.loc[x]["averageRating"]))
df_titles['numVotes'] = (df_titles.index.map(lambda x: df_title_ratings.loc[x]["numVotes"]))

In [9]:
df_title_akas = df_title_akas[(df_title_akas.index.isin(df_title_ratings.index))]

df_title_akas = df_title_akas[((df_title_akas.language == "EN") | (df_title_akas.language == "en") | (df_title_akas.language == "En"))]
df_title_akas

,ordering,title,region,language,types,attributes,isOriginalTitle
titleId,,,,,,,
tt0000574,4,The Story of the Kelly Gang,SG,en,imdbDisplay,\N,False
tt0000941,3,Love Crazy,XWW,en,\N,informal literal title,False
tt0001122,4,The Red Inn,XWW,en,\N,\N,False
tt0001175,4,Camille,XWW,en,\N,\N,False
tt0001338,2,A Night in May,XWW,en,\N,\N,False
...,...,...,...,...,...,...,...
tt9916362,13,Coven of Sisters,XWW,en,alternative,\N,False
tt9916362,3,Coven,CA,en,imdbDisplay,\N,False
tt9916362,4,Coven,EG,en,imdbDisplay,\N,False


In [10]:
df_titles = df_titles[(df_titles.startYear >= MOVIES_FROM_YEAR) & (df_titles.startYear <= MOVIES_TO_YEAR)]
df_titles = df_titles[(df_titles.index.isin(df_title_akas.index))]
df_titles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 97819 entries, tt0015414 to tt9916706
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   titleType       97819 non-null  string 
 1   originalTitle   97819 non-null  string 
 2   isAdult         97819 non-null  string 
 3   startYear       97819 non-null  Int64  
 4   runtimeMinutes  93140 non-null  Float64
 5   genres          97819 non-null  string 
 6   averageRating   97819 non-null  string 
 7   numVotes        97819 non-null  string 
dtypes: Float64(1), Int64(1), string(6)
memory usage: 6.9 MB


In [11]:
df_title_akas = []
df_title_ratings = []
df_titles = df_titles.sort_values(by=['numVotes' , 'averageRating'], ascending=False)
df_titles.head(10)

,titleType,originalTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes
tconst,,,,,,,,
tt0102307,movie,Light Sleeper,0,1992,103.0,"Crime,Drama",6.9,9996
tt10483386,movie,Mumbai Saga,0,2021,135.0,"Action,Crime,History",5.8,9992
tt6556670,movie,Qu'est-ce qu'on a encore fait au bon Dieu?,0,2019,99.0,"Comedy,Drama",6.1,9991
tt6769280,movie,Time Freak,0,2018,104.0,"Comedy,Drama,Romance",5.8,9991
tt0383028,movie,"Synecdoche, New York",0,2008,124.0,Drama,7.5,99900
tt1930294,movie,Black Rock,0,2012,83.0,"Horror,Thriller",4.8,9990
tt0342180,movie,A Certain Kind of Death,0,2003,69.0,Documentary,7.4,999
tt11394650,movie,On the Record,0,2020,95.0,Documentary,7.2,999
tt10785618,movie,Adolescentes,0,2019,135.0,Documentary,7.1,999


In [12]:
df_title_principals = []
df_title_principals = pd.read_csv(URL_TITLE_PRINCIPAL, compression='gzip', sep='\t', dtype={
    'tconst': 'string',
    'ordering': 'int64',
    'nconst': 'string',
    'category': 'string',
    'job': 'string'
}, index_col='tconst')

df_title_principals["category"] = df_title_principals["category"].replace("\\N", "")
df_title_principals["job"] = df_title_principals["job"].replace("\\N", "")
df_title_principals["characters"] = df_title_principals["characters"].replace("\\N", "")

In [13]:
df_title_principals = df_title_principals[(df_title_principals.category.isin(['actor', 'actress', 'director', 'producer']))]

#df_title_principals_ENMovie_CAT = df_title_principals_ENMovie_CAT[(((
#    (df_title_principals_ENMovie_CAT.category == 'actor') | 
#    (df_title_principals_ENMovie_CAT.category == 'actress')) &  
#    (df_title_principals_ENMovie_CAT.characters != '\\N')) | 
#    (df_title_principals_ENMovie_CAT.category.isin(['director', 'producer'])))]

df_title_principals.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51603419 entries, tt0000001 to tt9916880
Data columns (total 5 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   ordering    int64 
 1   nconst      string
 2   category    string
 3   job         string
 4   characters  object
dtypes: int64(1), object(1), string(3)
memory usage: 2.3+ GB


In [ ]:
df_title_principals_ENMovie = df_title_principals[(df_title_principals.index.isin(df_titles.index))]
df_title_principals_ENMovie.info()
df_title_principals = []

<class 'pandas.core.frame.DataFrame'>
Index: 1085969 entries, tt0015414 to tt9916706
Data columns (total 5 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   ordering    1085969 non-null  int64 
 1   nconst      1085969 non-null  string
 2   category    1085969 non-null  string
 3   job         1085969 non-null  string
 4   characters  1085969 non-null  object
dtypes: int64(1), object(1), string(3)
memory usage: 49.7+ MB


In [15]:
df_title_principals_ENMovie.head(10)


,ordering,nconst,category,job,characters
tconst,,,,,
tt0015414,3,nm0615736,director,,
tt0015414,4,nm0615736,producer,producer,
tt0035423,1,nm0000212,actress,,"[""Kate McKay""]"
tt0035423,2,nm0413168,actor,,"[""Leopold""]"
tt0035423,3,nm0000630,actor,,"[""Stuart Besser""]"
tt0035423,4,nm0005227,actor,,"[""Charlie McKay""]"
tt0035423,5,nm0005169,actress,,"[""Darci""]"
tt0035423,6,nm0925966,actor,,"[""J.J. Camden""]"
tt0035423,7,nm0925717,actor,,"[""Uncle Millard""]"


In [16]:
df_names = []
df_names = pd.read_csv(
    URL_ACTOR_BASICS, compression='gzip', 
    sep='\t', index_col='nconst',
    dtype={
        'nconst' : 'string',
        'primaryName' : 'string',
        'birthYear' : 'string',
        'deathYear' : 'string',
        'primaryProfession':'string',
        'knownForTitles' : 'string'
        })

df_names["primaryProfession"] = df_names["primaryProfession"].replace("\\N", "")
df_names["birthYear"] = df_names["birthYear"].replace("\\N", "")
df_names["birthYear"] = pd.to_numeric(df_names['birthYear'], downcast='integer' ,errors='coerce')
df_names["deathYear"] = df_names["deathYear"].replace("\\N", "")
df_names["deathYear"] = pd.to_numeric(df_names['deathYear'], downcast='integer', errors='coerce')

In [17]:
df_names.head(10)

,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
nconst,,,,,
nm0000001,Fred Astaire,1899,1987,"actor,miscellaneous,producer","tt0050419,tt0072308,tt0053137,tt0027125"
nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0117057,tt0038355"
nm0000003,Brigitte Bardot,1934,<NA>,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
nm0000004,John Belushi,1949,1982,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0069467,tt0083922,tt0050976"
nm0000006,Ingrid Bergman,1915,1982,"actress,producer,soundtrack","tt0034583,tt0036855,tt0038109,tt0038787"
nm0000007,Humphrey Bogart,1899,1957,"actor,producer,miscellaneous","tt0034583,tt0037382,tt0042593,tt0043265"
nm0000008,Marlon Brando,1924,2004,"actor,director,writer","tt0078788,tt0068646,tt0047296,tt0070849"
nm0000009,Richard Burton,1925,1984,"actor,producer,director","tt0061184,tt0087803,tt0059749,tt0057877"


In [18]:
df_names = df_names[df_names.index.isin(df_title_principals_ENMovie.nconst)]
df_title_principals_ENMovie = df_title_principals_ENMovie[df_title_principals_ENMovie.nconst.isin(df_names.index)]
df_movies = df_title_principals_ENMovie
df_title_principals_ENMovie = []

In [19]:
df_movies.loc[:, 'personName'] = df_movies.nconst.apply(lambda x: df_names.loc[x]['primaryName'])
df_movies.loc[:, 'birthYear'] = df_movies.nconst.apply(lambda x: df_names.loc[x]['birthYear'])
df_movies.loc[:, 'deathYear'] = df_movies.nconst.apply(lambda x: df_names.loc[x]['deathYear'])
df_movies.loc[:, 'primaryProfession'] = df_movies.nconst.apply(lambda x: df_names.loc[x]['primaryProfession'])
df_names = []

In [ ]:
df_movies.loc[:, 'originalTitle'] = df_movies.index.map(lambda x: df_titles.loc[x]['originalTitle'])
df_movies.loc[:, 'isAdult'] = df_movies.index.map(lambda x: df_titles.loc[x]['isAdult'])
df_movies.loc[:, 'startYear'] = df_movies.index.map(lambda x: df_titles.loc[x]['startYear'])
df_movies.loc[:, 'runtimeMinutes'] = df_movies.index.map(lambda x: df_titles.loc[x]['runtimeMinutes'])
df_movies.loc[:, 'genres'] = df_movies.index.map(lambda x: df_titles.loc[x]['genres'])
df_movies.loc[:, 'averageRating'] = df_movies.index.map(lambda x: df_titles.loc[x]['averageRating'])
df_movies.loc[:, 'numVotes'] = df_movies.index.map(lambda x: df_titles.loc[x]['numVotes'])
df_titles = []

In [ ]:
df_movies.sort_values(by=['numVotes' , 'averageRating'], ascending=False)
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1085963 entries, tt0015414 to tt9916706
Data columns (total 16 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   ordering           1085963 non-null  int64 
 1   nconst             1085963 non-null  string
 2   category           1085963 non-null  string
 3   job                1085963 non-null  string
 4   characters         1085963 non-null  object
 5   personName         1085963 non-null  object
 6   birthYear          523159 non-null   object
 7   deathYear          51651 non-null    object
 8   primaryProfession  1085963 non-null  object
 9   originalTitle      1085963 non-null  string
 10  isAdult            1085963 non-null  string
 11  startYear          1085963 non-null  int64 
 12  runtimeMinutes     1041405 non-null  object
 13  genres             1085963 non-null  string
 14  averageRating      1085963 non-null  string
 15  numVotes           1085963 non-null  string


In [ ]:
save_movie_title_to_csv(df_movies, f"imdb-movies-{MOVIES_FROM_YEAR}-{MOVIES_TO_YEAR}-years.csv")
